In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pl
import re
from sklearn.metrics import classification_report
import bs4
from collections import defaultdict

WORKING_DIR='.'
DEVICE = 'cuda'

In [18]:
dataset_educ = pd.read_json(f'{WORKING_DIR}/Dataset_educ_1.1.json', orient = 'index')
dataset_div = pd.read_json(f'{WORKING_DIR}/Dataset_div2_final.json', orient = 'index')

In [19]:
regex_link_ful = re.compile('<a href.*\/contest/.*/submission/.*<\/a>')
code_regex = re.compile('<code>(\s|.)*?<\/code>')
def preprocess_for_transfomers(texts, problems):
  preprocessed_texts = []
  for t, p in zip(texts, problems):
    t_codes = code_regex.sub(' (code) ', t)
    t_link = regex_link_ful.sub(f' (link to problem {p}) ', t_codes)
    bs = bs4.BeautifulSoup(t_link)
    preprocessed_texts.append(bs.text)
  
  return preprocessed_texts

dataset_educ.loc[~(dataset_educ['label'] == 'Irrelevant'), 'label'] = 'Relevant'
preprocessed_text = preprocess_for_transfomers(dataset_educ['text'], dataset_educ['problem'])
dataset_educ["preprocessed_text"] = preprocessed_text

dataset_div.loc[~(dataset_div['label'] == 'Irrelevant'), 'label'] = 'Relevant'
preprocessed_text = preprocess_for_transfomers(dataset_div['text'], dataset_div['problem'])
dataset_div["preprocessed_text"] = preprocessed_text

In [20]:
TREE_FATHER_PATH_LENGTH = 3

def compute_father_indices_pd(df):
  father_tree = {}

  for _,com in df.iterrows():
    father_tree[com.id] = com.father_id

  comment_father_indices = {}

  for df_index,com in df.iterrows():
      indices = []
      last_ind = com.id
      for i in range(TREE_FATHER_PATH_LENGTH):
        if(father_tree[last_ind] not in father_tree):
          indices.append(-1)
          continue
        if(last_ind != -1):
          last_ind = father_tree[last_ind]
        
        if(last_ind != -1):
          indices.append(df[df.id == last_ind].index.values[0])
        else:
          indices.append(last_ind)
      indices.reverse()
      comment_father_indices[df_index] = indices
  return comment_father_indices

father_indices_educ = compute_father_indices_pd(dataset_educ)
father_indices_div = compute_father_indices_pd(dataset_div)

In [21]:
from embeddings_generation import TokenizedDataset, LayerEMBTokenEmbeddingGeneration
from embeddings_generation.utils import *

huggingface_model_name = "roberta-128-base"
huggingface_model_name_alias = huggingface_model_name.split("/")[-1] + "_educ"

if(not embedding_already_persisted(huggingface_model_name_alias)):
    persist_embeddings(dataset_educ["preprocessed_text"], huggingface_model_name, huggingface_model_name_alias, dataset_educ.index)
    
embeddings_educ = load_embeddings(huggingface_model_name_alias)

huggingface_model_name_alias_div = huggingface_model_name.split("/")[-1] +'_div'
if(not embedding_already_persisted(huggingface_model_name_alias_div)):
    persist_embeddings(dataset_div["preprocessed_text"], huggingface_model_name, huggingface_model_name_alias_div, dataset_div.index)
embeddings_div = load_embeddings(huggingface_model_name_alias_div)

In [22]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import random
RANDOM_SEED = 443
DEVICE = "cuda"

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

groups = list(dataset_educ.groupby(lambda k : k.split("?")[0]))
random.shuffle(groups)

train_groups = groups[:10]
validation_groups = groups[10:13]
test_groups = groups[13:]

train_educ_dataset= dataset_educ.loc[[idx for _, g in train_groups for idx in g.index.tolist()]]
val_educ_dataset = dataset_educ.loc[[idx for _, g in validation_groups for idx in g.index.tolist()]]
test_educ_dataset = dataset_educ.loc[[idx for _, g in test_groups for idx in g.index.tolist()]]

from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
train_encoded_labels = labelEncoder.fit_transform(train_educ_dataset["label"])
val_encoded_labels = labelEncoder.fit_transform(val_educ_dataset["label"])
test_encoded_labels = labelEncoder.fit_transform(test_educ_dataset["label"])

div_encoded_labels = labelEncoder.fit_transform(dataset_div["label"])

In [23]:
from augmentation_generation import get_weak_augmented_text
from augmentation_generation.utils import persist_augmentations, load_augmentations
NUMBER_AUGMENTATIONS = 6

huggingface_model_name_aug_alias = huggingface_model_name_alias + f'_aug_{NUMBER_AUGMENTATIONS}'
if(not embedding_already_persisted(huggingface_model_name_aug_alias)):
    augmented_train_text = get_weak_augmented_text(train_educ_dataset['preprocessed_text'], NUMBER_AUGMENTATIONS)
    augmented_train_text = [a_text for a_text_list in augmented_train_text for a_text in a_text_list]
    augmented_indices = []

    for idx in train_educ_dataset.index:
        for aug_idx in range(NUMBER_AUGMENTATIONS):
            augmented_indices.append(idx + '__' + str(aug_idx))

    persist_embeddings(augmented_train_text, huggingface_model_name, huggingface_model_name_aug_alias, augmented_indices)
    persist_augmentations(augmented_train_text, huggingface_model_name_aug_alias + "__text", augmented_indices)
    augmented_father_indices = {}
    for k in train_educ_dataset.index:
        v = father_indices_educ[k]
        for aug_ind in range(NUMBER_AUGMENTATIONS):
            aug_k = k + "__" + str(aug_ind)
            aug_s_f_indices = []
            for f_ind in father_indices_educ[k]:
                if(f_ind == -1):
                    aug_s_f_indices.append(-1)
                else:
                    random_aug_index = np.random.choice(range(NUMBER_AUGMENTATIONS))
                    aug_s_f_indices.append(f_ind + "__" + str(random_aug_index))
            augmented_father_indices[aug_k] = aug_s_f_indices

    persist_augmentations(augmented_father_indices.values(), huggingface_model_name_aug_alias + "__fathers", augmented_father_indices.keys())

augmented_train_embeddings = load_embeddings(huggingface_model_name_aug_alias)
augmented_train_text = load_augmentations(huggingface_model_name_aug_alias + "__text")
augmented_father_indices = load_augmentations(huggingface_model_name_aug_alias + "__fathers")

train_educ_aug_dataset = train_educ_dataset.copy()
train_educ_aug_dataset["aug_text"] = None

train_educ_aug_dataset_dict = train_educ_aug_dataset.to_dict(orient='index')

for aug_key in augmented_train_embeddings.keys():
    data = train_educ_dataset.loc[aug_key.split("__")[0]].copy()
    data["aug_text"] = augmented_train_text[aug_key]
    train_educ_aug_dataset_dict[aug_key] = data.to_dict()

train_educ_aug_dataset = pd.DataFrame.from_dict(train_educ_aug_dataset_dict, orient = 'index')
train_aug_encoded_labels = labelEncoder.fit_transform(train_educ_aug_dataset["label"])


In [24]:
for k,v in augmented_train_embeddings.items():
    embeddings_educ[k] = v
for k,v in augmented_father_indices.items():
    father_indices_educ[k] = v

In [25]:
class BlogCommentDataset(Dataset):
    def __init__(self, dataset:pd.DataFrame, embeddings:dict, father_indices:dict, num_last_layers_embeddings_agg ,labels):
        self.dataset = dataset
        self.embeddings = embeddings
        self.father_indices = father_indices
        self.num_last_layers_embeddings_agg = num_last_layers_embeddings_agg
        self.labels = labels
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        df_index = self.dataset.index[index]
        embedding = self._agg_emb(self.embeddings[df_index])
        fathers = self.father_indices[df_index]

        embedding_fathers = []
        masks = []
        for f_idx in fathers:
            if(f_idx == -1):
                embedding_fathers.append(torch.zeros(embedding.shape[0]))
                masks.append(1)
            else:
                embedding_fathers.append(self._agg_emb(self.embeddings[f_idx]))
                masks.append(0)

        embedding_fathers = torch.stack(embedding_fathers)
        return embedding.to(DEVICE), embedding_fathers.to(DEVICE), torch.tensor(masks, dtype = torch.float32).to(DEVICE), torch.tensor(self.labels[index], dtype = torch.long).to(DEVICE)
    
    def _agg_emb(self, embedding):
        embedding = np.array(embedding, dtype=np.float32)
        embedding = embedding[0, -self.num_last_layers_embeddings_agg:, :].mean(0)
        return torch.from_numpy(embedding)

train_torch_dataset = BlogCommentDataset(train_educ_aug_dataset, embeddings_educ, father_indices_educ, 3, train_aug_encoded_labels)
train_torch_dataloader = DataLoader(train_torch_dataset, 2, shuffle=True)

val_torch_dataset = BlogCommentDataset(val_educ_dataset, embeddings_educ, father_indices_educ, 3, val_encoded_labels)
val_torch_dataloader = DataLoader(val_torch_dataset, 2, shuffle=False)

test_torch_dataset = BlogCommentDataset(test_educ_dataset, embeddings_educ, father_indices_educ, 3, test_encoded_labels)
test_torch_dataloader = DataLoader(test_torch_dataset, 2, shuffle=False)

div_dataset_torch = BlogCommentDataset(dataset_div, embeddings_div, father_indices_div, 3, div_encoded_labels)
div_dataloader_torch = DataLoader(div_dataset_torch, 2, shuffle=False)

In [26]:
from tqdm import tqdm
import math
class CommentClassificationModel(torch.nn.Module):
    def __init__(self, nrLabels):
        super(CommentClassificationModel, self).__init__()
        self.k = torch.nn.Linear(768, 128)
        self.q = torch.nn.Linear(768, 128)
        self.v = torch.nn.Linear(768, 128) 

        self.comment_proj = torch.nn.Linear(768, 128) 
        self.relu = torch.nn.ReLU()
        self.output = torch.nn.Linear(128, nrLabels) 
        self.hidden1 = torch.nn.Linear(128 * 2, 128)  
        self.dropout = torch.nn.Dropout(0.7)  

    def forward(self, x, fathers_x, mask):
        key = self.k(x)
        queries = self.q(fathers_x)
        values = self.v(fathers_x)

        key = torch.unsqueeze(key, -1)

        e_t = torch.bmm(queries, key)  / math.sqrt(128)
        e_t = torch.squeeze(e_t, -1)

        e_t = e_t + mask * -2e9
        a_t = torch.nn.Softmax()(e_t)

        a_t = torch.unsqueeze(a_t, -1)
        average_att = torch.bmm(a_t.permute(0, 2, 1), values)
        average_att = average_att.squeeze(1)
        average_att = average_att

        com_proj = self.comment_proj(x)

        h1 = self.dropout(self.relu(torch.cat([average_att, com_proj], -1)))
        h2 = self.dropout(self.relu(self.hidden1(h1)))

        return self.output(h2)

comment_classification_Model = CommentClassificationModel(len(labelEncoder.classes_))
comment_classification_Model.to(DEVICE)


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(comment_classification_Model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,len(train_torch_dataloader) * 5, 2)
nr_epochs = 50
current_step = 0
best_model_loss = 1e9
for epoch in range(nr_epochs):
  pbar_training = tqdm(train_torch_dataloader)
  training_average_loss = 0
  training_nr_batches = 0
  comment_classification_Model.train()
  iters = len(pbar_training)
  
  for embeddings_batch, embedding_fathers, mask_batch, labels_batch in pbar_training:
    optimizer.zero_grad()
    yhat = comment_classification_Model(embeddings_batch, embedding_fathers, mask_batch)
    loss = criterion(yhat, labels_batch)
    loss.backward()
    optimizer.step()
    pbar_training.set_postfix({'loss': loss.cpu().detach().numpy()})

    training_average_loss += loss.cpu().detach().numpy()
    training_nr_batches+=1
    current_step+=1
    scheduler.step()

  pbar_validation = tqdm(val_torch_dataloader)

  validation_average_loss = 0
  validation_nr_batches = 0
  comment_classification_Model.eval()
  for embeddings_batch, embedding_fathers, mask_batch, labels_batch in pbar_validation:
    with torch.no_grad():
      yhat = comment_classification_Model(embeddings_batch, embedding_fathers, mask_batch)
      loss = criterion(yhat, labels_batch)

      validation_average_loss += loss.cpu().detach().numpy()
      validation_nr_batches+=1

      pbar_validation.set_postfix({'loss': loss.cpu().detach().numpy()})
      
  print(f'Epoch {epoch + 1} has training loss: {training_average_loss / training_nr_batches}')
  print(f'Epoch {epoch + 1} has validation loss: {validation_average_loss / validation_nr_batches}')

  if(validation_average_loss / validation_nr_batches < best_model_loss):
     best_model_loss = validation_average_loss / validation_nr_batches
     print(f'Best loss at epoch {epoch}')
     torch.save(comment_classification_Model, f'best_models/{huggingface_model_name_aug_alias}.pkl')


  0%|          | 0/2069 [00:00<?, ?it/s]

C:\Users\Xzzyaa23\AppData\Local\Temp\ipykernel_26404\620435829.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t = torch.nn.Softmax()(e_t)
100%|██████████| 93/93 [00:00<00:00, 196.61it/s, loss=0.49709302]


Epoch 1 has training loss: 0.6331032526014493
Epoch 1 has validation loss: 0.43842838400153705
Best loss at epoch 0


100%|██████████| 93/93 [00:00<00:00, 187.11it/s, loss=0.49642748]


Epoch 2 has training loss: 0.49420010313555873
Epoch 2 has validation loss: 0.41442791176258875
Best loss at epoch 1


...

Epoch 48 has training loss: 0.308696353722555
Epoch 48 has validation loss: 0.49348517919587653


100%|██████████| 93/93 [00:00<00:00, 186.38it/s, loss=0.8473941]    


Epoch 49 has training loss: 0.3340764903551006
Epoch 49 has validation loss: 0.5943419340765494


100%|██████████| 93/93 [00:00<00:00, 200.53it/s, loss=0.42786884]   

Epoch 50 has training loss: 0.30144480315810707
Epoch 50 has validation loss: 0.49898090126384886


In [27]:
comment_classification_Model= torch.load(f'best_models/{huggingface_model_name_aug_alias}.pkl')
comment_classification_Model.to(DEVICE)
comment_classification_Model.eval()
predictions = []
pbar_validation = tqdm(val_torch_dataloader)
for embeddings_batch, embedding_fathers, mask_batch, labels_batch in pbar_validation:
  predictions.append(comment_classification_Model(embeddings_batch, embedding_fathers, mask_batch))

predictions = np.stack([e for b in predictions for e in b.cpu().detach().numpy()])

  0%|          | 0/93 [00:00<?, ?it/s]

C:\Users\Xzzyaa23\AppData\Local\Temp\ipykernel_26404\620435829.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t = torch.nn.Softmax()(e_t)
100%|██████████| 93/93 [00:00<00:00, 271.93it/s]


In [28]:
predictions_softmax = torch.nn.Softmax(dim=1)(torch.from_numpy(predictions)).detach().numpy()

predictions_indices = np.argmax(predictions_softmax, axis = 1)
predicted_normal_labels = labelEncoder.inverse_transform(predictions_indices.ravel())

from sklearn.metrics import classification_report

print(classification_report(val_educ_dataset["label"], predicted_normal_labels))

              precision    recall  f1-score   support

  Irrelevant       0.83      0.65      0.73        74
    Relevant       0.80      0.91      0.85       112

    accuracy                           0.81       186
   macro avg       0.81      0.78      0.79       186
weighted avg       0.81      0.81      0.80       186



In [29]:
predictions = []
pbar_test = tqdm(test_torch_dataloader)
for embeddings_batch, embedding_fathers, mask_batch, labels_batch in pbar_test:
  predictions.append(comment_classification_Model(embeddings_batch, embedding_fathers, mask_batch))

predictions = np.stack([e for b in predictions for e in b.cpu().detach().numpy()])

  0%|          | 0/80 [00:00<?, ?it/s]C:\Users\Xzzyaa23\AppData\Local\Temp\ipykernel_26404\620435829.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t = torch.nn.Softmax()(e_t)
100%|██████████| 80/80 [00:00<00:00, 99.63it/s] 


In [30]:
predictions_softmax = torch.nn.Softmax(dim=1)(torch.from_numpy(predictions)).detach().numpy()

predictions_indices = np.argmax(predictions_softmax, axis = 1)
predicted_normal_labels = labelEncoder.inverse_transform(predictions_indices.ravel())

from sklearn.metrics import classification_report

print(classification_report(test_educ_dataset["label"], predicted_normal_labels))

              precision    recall  f1-score   support

  Irrelevant       0.87      0.80      0.83        74
    Relevant       0.84      0.90      0.87        86

    accuracy                           0.85       160
   macro avg       0.85      0.85      0.85       160
weighted avg       0.85      0.85      0.85       160



In [31]:
predictions = []
pbar_div = tqdm(div_dataloader_torch)
for embeddings_batch, embedding_fathers, mask_batch, labels_batch in pbar_div:
  predictions.append(comment_classification_Model(embeddings_batch, embedding_fathers, mask_batch))

predictions = np.stack([e for b in predictions for e in b.cpu().detach().numpy()])

  0%|          | 0/252 [00:00<?, ?it/s]C:\Users\Xzzyaa23\AppData\Local\Temp\ipykernel_26404\620435829.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  a_t = torch.nn.Softmax()(e_t)
100%|██████████| 252/252 [00:00<00:00, 283.47it/s]


In [32]:
predictions_softmax = torch.nn.Softmax(dim=1)(torch.from_numpy(predictions)).detach().numpy()

predictions_indices = np.argmax(predictions_softmax, axis = 1)
predicted_normal_labels = labelEncoder.inverse_transform(predictions_indices.ravel())

from sklearn.metrics import classification_report

print(classification_report(dataset_div["label"], predicted_normal_labels))

              precision    recall  f1-score   support

  Irrelevant       0.97      0.64      0.77       393
    Relevant       0.42      0.92      0.58       111

    accuracy                           0.70       504
   macro avg       0.69      0.78      0.68       504
weighted avg       0.85      0.70      0.73       504

